# Jupyter Notebook Gruppe Tesla/Musk

Teammitglieder: Falk Jaresch, Michael Foeger, Paula Engelberg, Christoph Fink.

#### Rollenaufteilung:

- Datenrecherche und Ideenfindung - Gruppe
- Suche der passenden Datenquellen - Gruppe
- Konvertierung von CSV in RDF - Foeger
- Kombination der Datenquellen und Triple Store implementation - Engelberg
- Anfragen an die Datenquellen - Fink
- Erstellen einer Jekyll Seite - Jaresch

- Extra: Datenbearbeitung in OpenRefine (Datum) - Fink

#### Probleme:
Beim Erstellen der Queries für Aktien-Tagespreise an den bestimmten Tagen war es schwierig per Syntax die Formatierung der ersten Spalte (Datum) anzupassen.
Erst nach viel Recherche und Rücksprache mit Herrn Arndt war es klar, dass der einfachste Weg bei beiden Datenquellen eine einheitliche Formattierung des Datums zu erhalten (xsd:dateTime) war über OpenRefine.
Wir haben die Tesla-Stock CSV Datei in OpenRefine reingeladen und dann per Java Befehl "value.toDate('mm/dd/yyyy').toString('YYYY-MM-DD')" das Datum von einem amerikanischen Format auf das korrekte übersetzt. Danach musste nur noch die CSV Datei erneut in ein RDF File transformiert werden.

#### Reflexion der Techniken in Bezug auf den Technologiestack
-

#### Einverständnis zur Weitergabe erteilt




## Projekt Ausarbeitung

Datenquellen:

- Elon Musk Tweets: https://www.kaggle.com/kulgen/elon-musks-tweets

- Tesla Stock Price: https://www.kaggle.com/rpaguirre/tesla-stock-price



### Import der Datenquelle

In [1]:
from rdflib import Graph
from pandas import DataFrame

### Serialisierung: Parsen von Turtle zu ntriple

In [2]:
g = Graph()
g.parse("tesla.ttl", format="ttl")
g.serialize(destination='tesla.nt', format="ntriples")

In [3]:
g.parse("tweets.ttl", format="ttl")
g.serialize(destination='tweets.nt', format='ntriples')

## Queries

### Query nach Aktienpreis (Trading Tag Ende)


In [15]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>       
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX sioc: <http://rdfs.org/sioc/ns#>
PREFIX dc:   <http://purl.org/dc/elements/1.1/>

SELECT ?date ?close

WHERE {
   ?tradingDay ex:close ?close ;
    ex:date ?date .

 }

ORDER BY DESC(?date)
""")

DataFrame(result, columns=result.vars)

,date,close
0,2017-01-31T09:00:00+00:00,251.929993
1,2017-01-30T09:00:00+00:00,250.630005
2,2017-01-28T09:00:00+00:00,249.990005
3,2017-01-27T09:00:00+00:00,246.229996
4,2017-01-27T09:00:00+00:00,252.949997
...,...,...
1684,2009-01-01T09:00:00+00:00,20.6
1685,2009-01-01T09:00:00+00:00,20.450001
1686,2009-01-01T09:00:00+00:00,21.41
1687,2009-01-01T09:00:00+00:00,21.959999


### Abfrage nach Tweet Inhalt

In [14]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>       
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX sioc: <http://rdfs.org/sioc/ns#>
PREFIX dc:   <http://purl.org/dc/elements/1.1/>

SELECT ?content

WHERE {
   <http://example.org/resource/elonmusk/tweet/Row47> <http://rdfs.org/sioc/ns#content> ?content 

 }

""")

DataFrame(result, columns=result.vars)

,content
0,@SmileSimplify Sorry about that. You're right....


### Test

In [62]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>       
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX sioc: <http://rdfs.org/sioc/ns#>
PREFIX dc:   <http://purl.org/dc/elements/1.1/>

SELECT ?created

WHERE {
   ?date dc:created ?created .

 }

ORDER BY DESC(?date)
""")

DataFrame(result, columns=result.vars)

,created
0,2017-01-26T03:06:06+00:00
1,2017-01-27T13:29:30+00:00
2,2017-01-27T19:43:35+00:00
3,2017-01-27T20:32:52+00:00
4,2017-01-28T02:18:44+00:00
...,...
3213,2017-01-25T18:57:33+00:00
3214,2017-08-25T22:17:49+00:00
3215,2017-09-29T02:32:54+00:00
3216,2017-09-29T10:44:54+00:00
